In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os

### Prepare data
df = pd.read_csv('interview_data.csv')


#Drop the interviews that are too long (after more than 1024 tokens, does not work)
df = df[df['text'].apply(lambda x: len(x.split(' ')) < 350)]

#Create a very small test set to compare generated text with the reality
test_set = df.sample(n = 200)
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

#For the test set only, keep last 20 words in a new column, then remove them from original column
test_set['True_end_interview'] = test_set['text'].str.split().str[-20:].apply(' '.join)
test_set['text'] = test_set['text'].str.split().str[:-20].apply(' '.join)

In [ ]:
class Interview(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['text']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
    
dataset = Interview(df['text'], truncate=True, gpt2_type="gpt2")      

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
model = train(dataset, model, tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


619it [00:42, 14.67it/s]


Training epoch 1
tensor(3.6669, device='cuda:0', grad_fn=<NllLossBackward0>)


619it [00:43, 14.11it/s]


...

619it [00:43, 14.20it/s]


Training epoch 3
tensor(2.8779, device='cuda:0', grad_fn=<NllLossBackward0>)


619it [00:43, 14.13it/s]


Training epoch 4
tensor(1.8520, device='cuda:0', grad_fn=<NllLossBackward0>)


619it [00:43, 14.27it/s]


In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'tokenizer': tokenizer,
    'model_name': 'gpt2'
}, 'gpt2_model.pth')

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list


In [ ]:
class Process(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []
        self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{text_input}<|endoftext|>")
            ))               
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
text_input='I want to get pucks in deep'
text_process = Process(text_input, truncate=True, gpt2_type="gpt2") 

tensor([   27,    91,    40,   765,   284,   651,   279,  6238,   287,  2769,
           91,    29,    40,   765,   284,   651,   279,  6238,   287,  2769,
        50256])


In [ ]:
def text_generation(test_data):

  x = generate(model.to('cpu'), tokenizer, test_data, entry_count=1)
    
  return x

#Run the functions to generate the lyrics
generated_lyrics = text_generation('Have a conversation with me')

print(generated_lyrics)

100%|██████████| 1/1 [00:10<00:00, 10.80s/it]

['Have a conversation with me about how things are going or what we are going through," Stern said in a phone interview on Tuesday.\n\nWeeks later, the two traded<|endoftext|>']


In [ ]:
test_set.head()

,index,RowId,team1,team2,date,name,job,text,True_end_interview
0,1035,1035,kings,devils,2012-06-10,anze kopitar,player,you failed a couple times now we all know how ...,everybody wants to play hockey this time of th...
1,490,490,devils,avalanche,2001-05-31,patrick roy,player,the puck was loose and i wanted to help the gu...,like the third goal by danny it was a beautifu...
2,1459,1459,predators,penguins,2017-05-29,nick bonino,player,i think that comes from experience i think goi...,a chances getting some pressure but not gettin...
3,1331,1331,penguins,red wings,2009-05-30,justin abdelkader,player,yeah ville made a good play behind the net and...,game because you never know when your next shi...
4,846,846,hurricanes,red wings,2002-06-04,dominik hasek,player,we can't be happy because we lost they didn't ...,know what happened in the overtime it happened...
